In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('full_scrape_1.csv')

In [3]:
df.shape

(1024524, 4)

In [4]:
df.head()

,Comment Text,Author,Subreddit,Post ID
0,"Just curious, but what triggered your feelings...",PDXorax,YangForPresidentHQ,ew5ohva
1,Christmas time put up a light-up Yang sign!,PDXorax,YangForPresidentHQ,ew5o8vv
2,"During the general election, get in contact wi...",PDXorax,YangForPresidentHQ,ew42bdx
3,"We need to understand as a country, that the m...",PDXorax,YangForPresidentHQ,evva7l8
4,"Part of the reason for this, is because we nee...",PDXorax,YangForPresidentHQ,evoo179


In [5]:
df.dtypes

Comment Text    object
Author          object
Subreddit       object
Post ID         object
dtype: object

In [6]:
sum_series = df.groupby(['Author','Subreddit']).size()

In [7]:
users = list(set([x for x in sum_series.index.get_level_values(0)]))

In [8]:
len(users)

1682

In [9]:
list_of_subs = ['nba','nfl','movies','MMA','CFB','PoliticalHumor','teenagers',
                'soccer','freefolk','gaming','hockey','Libertarian','gameofthrones','MLS',
               ' JoeRogan','conspiracy','marvelstudios','television','asoiaf','leagueoflegends',
                'atheism','samharris','science','fantasyfootball','Cricket','CHIBears','hiphopheads',
                'technology','survivor','canada','Futurology']

In [10]:
features = {}
political_dict={}
for user in users:
    feature_dict = {}
    for sub in sum_series[user].index:
        if sub in list_of_subs:
            feature_dict[sub]=sum_series[user][sub]
    if len(feature_dict)!=0:
        features[user]=feature_dict
        #create target series
        inner_dict = {}
        try:
            inner_dict['yang'] = sum_series[user]['YangForPresidentHQ']
        except:
            inner_dict['yang'] = 0
        try:
            inner_dict['bernie'] = sum_series[user]['SandersForPresident']
        except:
            inner_dict['bernie'] = 0
        try:
            inner_dict['bernie'] += sum_series[user]['wayofthebern']
        except:
            inner_dict['bernie'] += 0
        try:
            inner_dict['bernie'] += sum_series[user]['Political_Revolution']
        except:
            inner_dict['bernie'] += 0
        try:
            inner_dict['pete'] = sum_series[user]['Pete_Buttigieg']
        except:
            inner_dict['pete'] = 0
        try:
            inner_dict['kamala'] = sum_series[user]['Kamala']
        except:
            inner_dict['kamala'] = 0
        try:
            inner_dict['biden'] = sum_series[user]['JoeBiden']
        except:
            inner_dict['biden'] = 0
        political_dict[user]=inner_dict


In [11]:
df_feature = pd.DataFrame(features).T

In [12]:
df_target = pd.DataFrame(political_dict).T

In [13]:
df_feature.head()

,CFB,CHIBears,Cricket,Futurology,Libertarian,MLS,MMA,PoliticalHumor,asoiaf,atheism,...,movies,nba,nfl,samharris,science,soccer,survivor,technology,teenagers,television
_-__-__-__-__-_-_-__,NaN,NaN,NaN,8.0,3.0,NaN,NaN,7.0,NaN,NaN,...,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0
MorningPerson42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
RSSforTulsi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Shootypatootie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,4.0,3.0,NaN,NaN,NaN,2.0,NaN
BerryBomB101,NaN,NaN,NaN,4.0,2.0,NaN,NaN,NaN,NaN,NaN,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN


In [14]:
df_feature.fillna(0,inplace=True)

In [15]:
#add code to combine like minded subs
df_feature['got']=df_feature['gameofthrones']+df_feature['asoiaf']
df_feature['soccer_comb']=df_feature['MLS']+df_feature['soccer']
df_feature['football']=df_feature['nfl']+df_feature['CFB']+df_feature['CHIBears']

In [16]:
#remove above single subs
df_feature.drop(['gameofthrones','asoiaf','soccer','MLS','nfl','CFB','CHIBears'],axis=1,inplace=True)

In [17]:
df_feature = df_feature.div(df_feature.sum(axis=1), axis=0)

In [18]:
def func(row):
    total = sum(row)
    if (row['bernie']/total >= .8) & (row['bernie']>=4):
        return 'bernie'
    if (row['biden']/total >= .8) & (row['biden']>=4):
        return 'biden'
    if (row['kamala']/total >= .8) & (row['kamala']>=4):
        return 'kamala'
    if (row['pete']/total >= .8) & (row['pete']>=4):
        return 'pete'
    if (row['yang']/total >= .8) & (row['yang']>=4):
        return 'yang'
    return 'unknown'
    

In [19]:
df_target.to_csv('noerrored.csv')

In [ ]:
df_target['target'] = df_target.apply(func,axis=1)

In [ ]:
df_target.drop(['bernie','biden','kamala','pete','yang'],axis=1,inplace=True)

In [ ]:
df = pd.concat([df_feature,df_target],axis=1)

In [ ]:
df.shape

In [ ]:
df=df[df['target']!='unknown']

In [ ]:
df.shape

In [ ]:
X = df.drop(['target'],axis=1)

In [ ]:
y = df['target']

In [ ]:
y.value_counts()

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression(multi_class='multinomial',solver='lbfgs')

In [ ]:
lr.fit(X,y)

In [ ]:
lr.classes_

In [ ]:
pd.DataFrame(lr.coef_.T,X.columns,lr.classes_)

In [ ]:
#removing the biden and kamala members since they're counts are really low

In [ ]:
df2 = df[(df['target']!='kamala')&(df['target']!='biden')]

In [ ]:
df2.shape

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X2 = df2.drop(['target'],axis=1)

In [ ]:
y2 = df2['target']

In [ ]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, random_state=42)

In [ ]:
lr2 = LogisticRegression(multi_class='multinomial',solver='lbfgs')

In [ ]:
lr2.fit(X2_train,y2_train)

In [ ]:
lr2.score(X2_train,y2_train)

In [ ]:
lr2.score(X2_test,y2_test)

In [ ]:
pd.DataFrame(lr2.coef_.T,X2.columns,lr2.classes_)

In [ ]:
y2.value_counts(normalize=True)

In [ ]:
import numpy as np

In [ ]:
np.sqrt(752)

In [ ]:
#how much better are my results if I go yang and not yang

In [ ]:
df3 = df2.copy()

In [ ]:
df3.shape

In [ ]:
df3['target'].value_counts()

In [ ]:
def convertdf3(text):
    if text == 'yang':
        return 1
    else:
        return 0

In [ ]:
df3['target']=df3['target'].apply(convertdf3)

In [ ]:
df3['target'].value_counts()

In [ ]:
X3 = df3.drop(['target'],axis=1)

In [ ]:
y3 = df3['target']

In [ ]:
X3_train, X3_test, y3_train, y3_test = train_test_split(X3, y3, stratify = y3,random_state=42)

In [ ]:
lr3 = LogisticRegression()

In [ ]:
lr3.fit(X3_train,y3_train)

In [ ]:
lr3.score(X3_train,y3_train)

In [ ]:
lr3.score(X3_test,y3_test)

In [ ]:
y3.value_counts(normalize=True)

In [ ]:
#how about yang vs. bernie

In [ ]:
df4 = df2.copy()

In [ ]:
def convertdf4(text):
    if text == 'yang':
        return 1
    elif text =='bernie':
        return 0
    else:
        return 7

In [ ]:
df4['target']=df4['target'].apply(convertdf4)

In [ ]:
df4 = df4[df4['target']!=7]

In [ ]:
df4['target'].value_counts()

In [ ]:
X4 = df4.drop(['target'],axis=1)
y4 = df4['target']

In [ ]:
X4_train, X4_test, y4_train, y4_test = train_test_split(X4, y4, stratify = y4,random_state=42)

In [ ]:
lr4 = LogisticRegression()

In [ ]:
lr4.fit(X4_train,y4_train)

In [ ]:
lr4.score(X4_train,y4_train)

In [ ]:
lr4.score(X4_test,y4_test)

In [ ]:
y4.value_counts(normalize=True)